# Dataset Setup Test

In [1]:
from src.data.datasetup import AmazonDatasetSetup
import pandas as pd
import os

### Dataset Test 1

In [2]:
root="data/AmazonReviews"
os.makedirs(root, exist_ok=True)
datasetConfig = "src/data/datasetConfigAmazon.json"
datasetName = "AmazonAllBeautyDataset"

AmazonAllBeautyDataset = AmazonDatasetSetup(root, datasetConfig, datasetName, devCtrl=False)

Reading config file at src/data/datasetConfigAmazon.json
Dataset Config set as:
{'category': 'All_Beauty',
 'urls': {'interactionDataUrl': 'https://datarepo.eng.ucsd.edu/mcauley_group/data/amazon_2023/benchmark/5core/rating_only/',
          'metaDataUrl': 'https://datarepo.eng.ucsd.edu/mcauley_group/data/amazon_2023/raw/meta_categories/',
          'reviewDataUrl': 'https://datarepo.eng.ucsd.edu/mcauley_group/data/amazon_2023/raw/review_categories/'},
 'filterCriteria': {'requiredFields': {'parent_asin': True,
                                       'description': False,
                                       'images': False,
                                       'title': False,
                                       'average_rating': False,
                                       'features': False,
                                       'price': False,
                                       'details': False},
                    'filterItemsBasedOnFeatures': True,
                    

In [3]:
AmazonAllBeautyDataset.interactionData.user_id.value_counts()

AttributeError: 'AmazonDatasetSetup' object has no attribute 'interactionData'

In [ ]:
AmazonAllBeautyDataset.interactionData.parent_asin.value_counts()

In [ ]:
AmazonAllBeautyDataset.interactionData.shape

In [ ]:
AmazonAllBeautyDataset.interactionData

In [ ]:
testcsv = pd.read_csv("data/AmazonReviews/All_Beauty/raw/Interactions/All_Beauty_Preprocessed.csv.gz")
testcsv

### Dataset Test 2

In [ ]:
# Industrial
root="data/AmazonReviews"
os.makedirs(root, exist_ok=True)
datasetConfig = "src/data/datasetConfigAmazon.json"
datasetName = "AmazonIndustrialDataset"

AmazonIndustrialDataset = AmazonDatasetSetup(root, datasetConfig, datasetName, devCtrl=False)

In [ ]:
import pandas as pd
testcsv = pd.read_csv("data/AmazonReviews/Industrial_and_Scientific/raw/Interactions/Industrial_and_Scientific_Preprocessed.csv.gz")
testcsv

In [ ]:
testcsv.user_id.value_counts()

In [ ]:
testcsv.parent_asin.value_counts()

# Graph Datasets Test

In [4]:
# Industrial
root="data/AmazonReviews"
os.makedirs(root, exist_ok=True)
datasetConfig = "src/data/datasetConfigAmazon.json"
datasetName = "AmazonIndustrialDataset"

AmazonIndustrialDataset = AmazonDatasetSetup(root, datasetConfig, datasetName, devCtrl=False)

Reading config file at src/data/datasetConfigAmazon.json
Dataset Config set as:
{'category': 'Industrial_and_Scientific',
 'urls': {'interactionDataUrl': 'https://datarepo.eng.ucsd.edu/mcauley_group/data/amazon_2023/benchmark/5core/rating_only/',
          'metaDataUrl': 'https://datarepo.eng.ucsd.edu/mcauley_group/data/amazon_2023/raw/meta_categories/',
          'reviewDataUrl': 'https://datarepo.eng.ucsd.edu/mcauley_group/data/amazon_2023/raw/review_categories/'},
 'filterCriteria': {'requiredFields': {'parent_asin': True,
                                       'description': True,
                                       'images': True,
                                       'title': True,
                                       'average_rating': False,
                                       'features': False,
                                       'price': False,
                                       'details': False},
                    'filterItemsBasedOnFeatures': True,
        

In [5]:
from src.data.graphDatasets import AmazonGraphDataset
from src.data.utils import binarizeEdgesByThreshold

In [6]:
rating_threshold = 3  #@param {type: "integer"}: Ratings equal to or greater than 3 are positive items.

config_dict = {
    "num_samples_per_user": 500,
    "num_users": 200,

    "epochs": 100,
    "batch_size": 128,
    "lr": 0.001,
    "weight_decay": 0.1,

    "embedding_size": 64,
    "num_layers": 5,
    "K": 10,
    "mf_rank": 8,

    "minibatch_per_print": 100,
    "epochs_per_print": 1,

    "val_frac": 0.2,
    "test_frac": 0.1,

    "model_name": "model.pth"
}

In [7]:
amazonIndustrialDataset = AmazonGraphDataset(
    datasetSetup = AmazonIndustrialDataset,
    config_dict = config_dict,
    rating_threshold = rating_threshold,
    transform=binarizeEdgesByThreshold
)

In [8]:
amazonIndustrialDataset.processed_file_names

'data_amazon_Industrial_and_Scientific.pt'

In [9]:
amazonIndustrialDataset.process()

Processing finished.


In [ ]:
data = amazonIndustrialDataset.get()

train_mask, val_mask, test_mask = \
        amazonIndustrialDataset.train_val_test_split(val_frac=config_dict["val_frac"],
                                       test_frac=config_dict["test_frac"])

n_users = len(pd.Series(data["users"]).unique())
m_items = len(pd.Series(data["items"]).unique())
print(f"#Users: {n_users}")
print(f"#Items: {m_items}")

/Users/timthiele/Documents/Uni/Masterthesis/MLLM_RecSys/src/data/graphDatasets.py:114: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  data = torch.load(os.path.join(self.proc

AttributeError: 'numpy.ndarray' object has no attribute 'unique'